In [13]:
!pip install -qU openai-agents

/usr/lib/python3.11/pathlib.py:542: RuntimeWarning: coroutine 'main' was never awaited
  self._str = self._format_parsed_parts(self._drv, self._root,
/usr/lib/python3.11/pathlib.py:542: RuntimeWarning: coroutine 'AgentRunner.run' was never awaited
  self._str = self._format_parsed_parts(self._drv, self._root,
/usr/lib/python3.11/pathlib.py:540: RuntimeWarning: coroutine 'main' was never awaited
  return self._str


In [18]:
import nest_asyncio
nest_asyncio.apply()

In [14]:
import os
from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

In [19]:
import asyncio

from openai.types.responses import ResponseTextDeltaEvent

from agents import Agent, Runner, function_tool, RunContextWrapper
from dataclasses import dataclass

@function_tool
def get_weather(city:str):
  """Gets weather for a city"""
  return f"The weather in {city} is sunny"

@dataclass
class CustomContext():
  name:str
  age:int

context = CustomContext(name="Suhaib", age=18)

def custom_instructions(
    run_context: RunContextWrapper[CustomContext], agent: Agent[CustomContext]
) -> str:
    context = run_context.context
    return f"You are a specialized assistant for {context.name}"

async def main():
    agent = Agent(
        name="Assistant",
        instructions=custom_instructions,
        model="gpt-4.1",
        tools=[get_weather]

    )

    result = Runner.run_streamed(agent, input="What is the weather in New York?",context=context)
    async for event in result.stream_events():
        if event.type == "raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
            print(event.data.delta, end="", flush=True)
        elif event.type == "agent_updated_stream_event":
            print(f"Agent updated: {event.new_agent.name}")
        elif event.type == "run_item_stream_event":
          print(event.name)



asyncio.run(main())

/usr/lib/python3.11/tokenize.py:529: RuntimeWarning: coroutine 'main' was never awaited
  pseudomatch = _compile(PseudoToken).match(line, pos)


Agent updated: Assistant
tool_called
tool_output
The weather in New York is currently sunny. If you need more specific details like temperature or forecast, let me know!message_output_created
